In [2]:
from flask import Flask

In [ ]:
import random
from flask import Flask, jsonify

app = Flask(__name__)

def generate_list():
    my_list = ["item1", "item2", "item3"]
    return my_list

def welcome():
    m = "Aircraft Route Optimization with Genetic Algorithm!"
    return m

def generate_random_combinations():
    # List of countries
    countries = ['Multan', 'Islamabad', 'Peshawar', 'Karachi', 'Quetta']

    num_routes = random.randint(5, 10)
    random_routes = [random.sample(countries, len(countries)) for _ in range(num_routes)]

    combinations_dict = {}

    # Generate and store 3 combinations for each route
    for i, country_list in enumerate(random_routes, 1):
        combinations_dict[f"Route {i}"] = []

        comb = random.sample(country_list, 3)
        combinations_dict[f"Route {i}"].append(comb)

    return combinations_dict

@app.route("/")
def hello():
    # Call the welcome function to get the first heading
    headingline = welcome()

    my_list = generate_list()
    items = " -> ".join(my_list)

    # Generate random route combinations
    route_combinations = generate_random_combinations()

    # Convert route combinations to a formatted string
    route_combinations_str = "<br>".join([f"<b>{route}:</b> {', '.join(map(str, countries))}" for route, countries in route_combinations.items()])

    return f'''
    <html>
    <head>
    <title>Welcome to Aircraft Route Optimization with Genetic Algorithm!</title>
    <h style="font-size: 36px;"><strong>Welcome to Aircraft Route Optimization with Genetic Algorithm!</strong></h>

        <style>
            body {{
                background-color: green;
                font-family: Arial, sans-serif;
                text-align: center;
            }}
            ul {{
                list-style: none;
                padding: 0;
            }}
            .centered {{
                text-align: center;
                margin: 20px;
            }}
            .rounded-box {{
                background-color: white;
                border-radius: 10px;
                padding: 20px;
                display: inline-block;
            }}
            h1 {{
                color: green;
            }}
            h2 {{
                color: green;
            }}
            b {{
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <h1>{headingline}</h1>
        <div class="centered">
            <div class="rounded-box">
                <ul>
                    <li>{items}</li>
                </ul>
            </div>
            <h2>Random Route Combinations</h2>
            <p>{route_combinations_str}</p>
        </div>
    </body>
    </html>
    '''

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Oct/2023 23:39:31] "GET / HTTP/1.1" 200 -


In [ ]:
one min